# Performance des investissements

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

In [11]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

INTERET_LIVRET_A = config["INTERET_LIVRET_A"]
DATA_FOLDER = config["DATA_FOLDER"]
INFLATION = config["INFLATION"]
IMAGE_FOLDER = config["IMAGE_FOLDER"]

In [12]:
def calcul_pmv(row):
    if row["Type"] != "livret":
        return row["Valorisation"] - row["Apport"]
    else:
        return 0


def calcul_perf(row):
    if row["Type"] != "livret":
        return row["Plus-Moins Value"] / row["Valorisation"]
    else:
        return INTERET_LIVRET_A


def calcul_apport(df, actif):
    return df.loc[df["Actif"] == actif, "Montant"].sum()


def color_based_on_value(val):
    if val > INFLATION * 2:
        return "background-color: lightgreen"
    elif val > INFLATION:
        return "background-color: gold"
    else:
        return "background-color: salmon"


def my_print(df):
    return (
        df.drop(columns=["Type"])
        .style.format(
            {
                "Valorisation": "{:,.2f}€",
                "Apport": "{:,.2f}€",
                "Plus-Moins Value": "{:,.2f}€",
                "Performance": "{:.2%}",
            }
        )
        .hide()
    ).map(color_based_on_value, subset=["Performance"])

In [13]:
apport = pd.read_csv(DATA_FOLDER + "apport.csv")
apport["Date"] = pd.to_datetime(apport["Date"], dayfirst=True)
apport["Montant"] = pd.to_numeric(apport["Montant"])

In [14]:
patrimoine = pd.read_csv(DATA_FOLDER + "patrimoine.csv")
patrimoine["Valorisation"] = pd.to_numeric(patrimoine["Valorisation"])
apport_list = []
for actif in patrimoine["Actif"]:
    apport_list.append(calcul_apport(apport, actif))
patrimoine["Apport"] = apport_list

In [15]:
patrimoine["Plus-Moins Value"] = patrimoine.apply(calcul_pmv, axis=1)
patrimoine["Performance"] = patrimoine.apply(calcul_perf, axis=1)

In [ ]:
my_print(patrimoine)

In [17]:
patrimoine_melted = patrimoine.drop(
    columns=["Apport", "Type", "Valorisation", "Plus-Moins Value"]
).melt(id_vars="Actif", var_name="Type", value_name="Valeur")

In [18]:
plt.figure(figsize=(10, 6))
sns.barplot(data=patrimoine_melted, x="Actif", y="Valeur", hue="Type")

plt.title("Performance par Actif", fontsize=14)
plt.xlabel("Actif")
plt.ylabel("Valeur")
plt.legend(title="Type de mesure")

plt.savefig(IMAGE_FOLDER + "perf-par-actif.png")
plt.close()